<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTunning_Testing_For_MedalQADataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tunning: https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_Mistral_7b_hfdeployment_dataset_Medmcqa.ipynb

In [ ]:
!pip install colab-env --quiet
!pip install -U tensorboard-plugin-profile -q

In [ ]:
%rm -rf /content/tensorbard
%mkdir -p /content/tensorbard
%cd /content/tensorbard
!git lfs install
#https://huggingface.co/frankmorales2020/Mistral-7B-v0.1_McGill-MEDAL/tree/main
!git clone https://huggingface.co/frankmorales2020/Mistral-7B-v0.1_McGill-MEDAL

In [ ]:
# Load the TensorBoard notebook extension

%load_ext tensorboard
%tensorboard --logdir /content/tensorbard/Mistral-7B-v0.1_McGill-MEDAL/runs

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
#!pip install colab-env --quiet


!pip install huggingface_hub -q

In [ ]:
!pip install mistral_inference -q

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)


import torch
from peft import AutoPeftModelForCausalLM

peft_model_id = "frankmorales2020/Mistral-7B-v0.1_McGill-MEDAL"

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  #device_map=None,
  device_map="auto",
  torch_dtype=torch.float16,
  attn_implementation="flash_attention_2",
)


tokenizer = AutoTokenizer.from_pretrained(peft_model_id, use_fast=True)

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# load into pipeline
pipe = pipeline("document-question-answering", model=model, tokenizer=tokenizer)

In [ ]:
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map

prompt = 'to quantify the amount of CL NO harvested in modified RND'
outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

In [ ]:
outputs

[{'generated_text': 'to quantify the amount of CL NO harvested in modified RND systems the in vitro harvesting rate was determined in wells that were seeded with cells and coated with NF membrane the NO harvesting rate was also determined in vivo by measuring the net increase in cytoplasmic sodium concentration csa from to mmold a rate constant turnover t of h was calculated for the in vitro system the in vivo rate constant turnover was determined to be x gmin this in vivo value is close to the in vitro value and indicates that most of the collection occurs via the nakatpase in the intact renal tubule nephroselect'}]

In [ ]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
ganswer

'systems the in vitro harvesting rate was determined in wells that were seeded with cells and coated with NF membrane the NO harvesting rate was also determined in vivo by measuring the net increase in cytoplasmic sodium concentration csa from to mmold a rate constant turnover t of h was calculated for the in vitro system the in vivo rate constant turnover was determined to be x gmin this in vivo value is close to the in vitro value and indicates that most of the collection occurs via the nakatpase in the intact renal tubule nephroselect'

In [ ]:
prompt = '[INST] to quantify the amount of CL NO harvested in modified RND [/INST]'
outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [ ]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
ganswer

"'radical neck dissection'"

In [ ]:
prompt = 'to quantify the amount of CL NO harvested in modified RND'

outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
#print(f'{prompt}')
print(f'{ganswer}')

systems the in vitro harvesting rate was determined in wells that were seeded with cells and coated with NF membrane the NO harvesting rate was also determined in vivo by measuring the net increase in cytoplasmic sodium concentration csa from to mmold a rate constant turnover t of h was calculated for the in vitro system the in vivo rate constant turnover was determined to be x gmin this in vivo value is close to the in vitro value and indicates that most of the collection occurs via the nakatpase in the intact renal tubule nephroselect


In [ ]:
import colab_env
from datasets import load_dataset
print("Preprocessing dataset McGill-MEDAL")
dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")

Preprocessing dataset Emotion


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['abstract_id', 'text', 'location', 'label'],
    num_rows: 1000000
})

In [ ]:
nrec=10
dataset_final_Question=dataset['text'][0:nrec]
dataset_final_Answer=dataset['label'][0:nrec]
#dataset_final_Messages=dataset['messages'][0:nrec]

In [ ]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame

datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer

In [ ]:
dataset[6]

{'abstract_id': 13253027,
 'text': 'while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading

In [ ]:
datasetF

,Question,Answer
0,we developed an animal model of chronic allerg...,[functional residual capacity]
1,pyogenic granulomas represent the aquisition o...,[pyogenic granuloma]
2,the l immunotype lipopolysaccharide lps of nei...,[phosphorylethanolamine]
3,inotropic reserve identified by dobutamine or ...,[stress echocardiography]
4,pyridoxinedependent seizure is a rare autosoma...,[severe mental retardation]
5,a total of PA isolates were collected from GA ...,[metallobetalactamase]
6,while diminished ovarian reserve dor predicts ...,[antral follicle count]
7,we describe two cases of simple heterozygosity...,[thalassemia intermedia]
8,by means of EC and quantitative histomorphomet...,[electrochemical]
9,in the present study lithocholic acid lca meta...,[eye movement]


In [ ]:
from datasets import load_dataset
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")

In [ ]:
eval_dataset

Dataset({
    features: ['abstract_id', 'text', 'location', 'label'],
    num_rows: 1000000
})

In [ ]:
nrec=51
eval_dataset[nrec]

{'abstract_id': 12479069,
 'text': 'a highly efficient synthesis of planar chiral ferrocenes by enantioselective pdcatalyzed direct ch arylation from readily available starting materials under mild reaction conditions was developed up to yield ee the products can be easily transformed to the highly efficient planar Fc ligands which have demonstrated high efficiency in pdcatalyzed asymmetric allylic alkylation and amination reactions',
 'location': [40],
 'label': ['ferrocene']}

In [ ]:
eval_dataset[nrec]['text']

'a highly efficient synthesis of planar chiral ferrocenes by enantioselective pdcatalyzed direct ch arylation from readily available starting materials under mild reaction conditions was developed up to yield ee the products can be easily transformed to the highly efficient planar Fc ligands which have demonstrated high efficiency in pdcatalyzed asymmetric allylic alkylation and amination reactions'

In [ ]:
eval_dataset[nrec]['label']

['ferrocene']

https://nbviewer.org/github/frank-morales2020/MLxDL/blob/main/upload_model_hf.ipynb

In [ ]:
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map

#prompt = generation_pipeline.tokenizer.apply_chat_template(modified_conversation, tokenize=False, add_generation_prompt=True)

prompt =  eval_dataset[nrec]['text']
print(prompt)
outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

a highly efficient synthesis of planar chiral ferrocenes by enantioselective pdcatalyzed direct ch arylation from readily available starting materials under mild reaction conditions was developed up to yield ee the products can be easily transformed to the highly efficient planar Fc ligands which have demonstrated high efficiency in pdcatalyzed asymmetric allylic alkylation and amination reactions


In [ ]:
ganswer=outputs[0]['generated_text'][len(prompt):]
ganswer

" with chiral ferrocene ligands as chiral catalysts [/INST] 'ferrocene' "

In [ ]:
qc=str(ganswer).find('[/INST]')
ganswer=ganswer[qc+8:len(ganswer)]
print(ganswer)

'ferrocene' 


In [ ]:
import re

oanswer=eval_dataset[nrec]['label']
ganswer=outputs[0]['generated_text'][len(prompt):].strip()



qc=str(ganswer).find('[/INST]')
ganswer=ganswer[qc+8:len(ganswer)]

answer = re.sub("\[", "", str(oanswer))
oanswer = re.sub("\]", "", str(answer))
#oanswer = re.sub(" ", "", str(oanswer))

#oanswer=str(oanswer[1:len(oanswer)-1])

print(f"Question:\n{eval_dataset[nrec]['text']}")
print()
print(f"Original Answer:{oanswer}")
print()
print(f"Generated Answer:{ganswer}")

print()
if oanswer == ganswer:
   print('match')
   qc=1
else:
   print('no match')
   qc=0

Question:
a highly efficient synthesis of planar chiral ferrocenes by enantioselective pdcatalyzed direct ch arylation from readily available starting materials under mild reaction conditions was developed up to yield ee the products can be easily transformed to the highly efficient planar Fc ligands which have demonstrated high efficiency in pdcatalyzed asymmetric allylic alkylation and amination reactions

Original Answer:'ferrocene'

Generated Answer:'ferrocene'

match


In [ ]:
import re

def similar(oanswer,ganswer):
    if oanswer == ganswer:
      #print('match')
      qc=1
    else:
      #print('no match')
      qc=0
    return qc

In [ ]:
number_of_eval_samples = 10
eval_dataset.shuffle().select(range(number_of_eval_samples))

Dataset({
    features: ['abstract_id', 'text', 'location', 'label'],
    num_rows: 10
})

In [ ]:
from tqdm import tqdm
import re

# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")


def evaluate(sample):

    prompt = sample['text']
    answer=sample['label']

    if answer == None:
       answer_original = 'Not possible to get or use'

    answer_original = re.sub("\n", "", str(answer))


    if len(answer_original)==0:
        answer_original='Not possible to get or use'

    str_question= sample['text']

    prompt = re.sub("\n", "", str_question)

    #prompt = eval_dataset[rand_idx]['Question']
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)
    #prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    #outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt)+8:].strip()
    #print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")

    oanswer=sample['label']

    if oanswer == None:
       oanswer = 'Not possible to get or use'

    answer = re.sub("\[", "", str(oanswer))
    oanswer = re.sub("\]", "", str(answer))

    ganswer=outputs[0]['generated_text'][len(prompt)+8:].strip()
    qc=str(ganswer).find('[/INST]')

    #print(qc)
    if int(qc)<0:
       #print(f"Generated Answer:\n{ganswer}")
       ganswer=outputs[0]['generated_text'][len(prompt)+8:].strip()
    else:
       ganswer=ganswer[qc+8:len(ganswer)]


    ms = similar(str(oanswer), str(ganswer))

    # printing the result
    #print ("The similarity between 2 strings is : " + str(res))

    if  ms==1:
          #print ()
          #print ("The similarity between 2 strings is : " + str(res))
          #print(f"Question:\n{sample['text']}")
          #print(f"Original Answer:\n{answer_original}")
          #print(f"Generated Answer:\n{ganswer}")
          return 1
    else:
          #print ()
          #print(f"FULL Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")
          #print(f"Question:\n{sample['text']}")
          #print(f"Generated Answer:\n{ganswer}")
          #print(f"Original Answer:\n{oanswer}")
          #print()
          return 0
    #del ganswer
success_rate = []
number_of_eval_samples = 1000
# iterate over eval dataset and predict

#for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
for n in range(number_of_eval_samples):
    s=eval_dataset[n]
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

#print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")

In [ ]:
print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")